# Cálculo de ratios

Cálculo de los ratios financieros a partir de la información contable y bursátil disponible, que utilizaremos como variables explicativas del modelo

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 50

In [3]:
! ls -n ../tablas

total 33744
-rw-rw-r-- 1 1000 1000 12810971 sep  4 19:08 dataformodel.csv
-rw-rw-r-- 1 1000 1000    76982 sep  4 16:37 filteredcompanies.csv
-rw-rw-r-- 1 1000 1000 13719490 sep  4 16:37 mergedbalres.csv
-rw-rw-r-- 1 1000 1000  7921206 sep  4 18:58 registros_etiq.csv
-rw-rw-r-- 1 1000 1000     2057 ago 31 11:21 var_indice.csv
-rw-rw-r-- 1 1000 1000     1830 sep  4 18:58 var_indice_energy.csv
-rw-rw-r-- 1 1000 1000     1835 sep  4 18:58 var_indice_health.csv
-rw-rw-r-- 1 1000 1000     1836 sep  4 18:58 var_indice_ind.csv
-rw-rw-r-- 1 1000 1000     1761 sep  4 18:58 var_indice_tec.csv


In [4]:
data = pd.read_csv('../tablas/registros_etiq.csv', usecols=['Ticker', 'SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period',
       'Report Date', 'Publish Date', 'Restated Date', 'Shares (Basic)',
       'Shares (Diluted)', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity',
       'Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Depreciation & Amortization', 'Operating Income (Loss)',
       'Non-Operating Income (Loss)', 'Interest Expense, Net',
       'Pretax Income (Loss), Adj.', 'Abnormal Gains (Losses)',
       'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income', 'Net Income (Common)',
       'Periodo', 'Price_d', 'Price_d_100', 'Publish_Date+180', 'Price_d+180',
       'Price_d+180_100', 'Indice_d_100', 'Indice_d+180_100', 'Var_cotiz',
       'Var_index', 'Etiqueta','Sector'])

In [5]:
len(data)

12706

In [6]:
list(data.columns)

['Ticker',
 'SimFinId',
 'Currency',
 'Fiscal Year',
 'Fiscal Period',
 'Report Date',
 'Publish Date',
 'Restated Date',
 'Shares (Basic)',
 'Shares (Diluted)',
 'Cash, Cash Equivalents & Short Term Investments',
 'Accounts & Notes Receivable',
 'Inventories',
 'Total Current Assets',
 'Property, Plant & Equipment, Net',
 'Long Term Investments & Receivables',
 'Other Long Term Assets',
 'Total Noncurrent Assets',
 'Total Assets',
 'Payables & Accruals',
 'Short Term Debt',
 'Total Current Liabilities',
 'Long Term Debt',
 'Total Noncurrent Liabilities',
 'Total Liabilities',
 'Share Capital & Additional Paid-In Capital',
 'Treasury Stock',
 'Retained Earnings',
 'Total Equity',
 'Total Liabilities & Equity',
 'Sector',
 'Revenue',
 'Cost of Revenue',
 'Gross Profit',
 'Operating Expenses',
 'Selling, General & Administrative',
 'Research & Development',
 'Depreciation & Amortization',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Interest Expense, Net',
 'Pretax Incom

### Capitalización bursátil

In [7]:
data['Capit']=data['Shares (Basic)']*data['Price_d']
len(data[data['Capit'].isnull()])

62

### PER

In [8]:
data['PER'] = data['Capit']/data['Net Income (Common)']
len(data[data['PER'].isnull()])

62

### Price / Book Value

In [9]:
data['PBC'] = data['Capit']/data['Total Equity']
len(data[data['PBC'].isnull()])

62

### ROE

In [10]:
data['ROE'] = data['Net Income (Common)']/data['Total Equity']
len(data[data['ROE'].isnull()])

0

### ROA

In [11]:
data['ROA'] = data['Net Income (Common)']/data['Total Assets']
len(data[data['ROA'].isnull()])

0

### Deuda total / FFPP

In [12]:
data['Deuda_FFPP'] = (data['Short Term Debt'] + data['Long Term Debt'])/data['Total Equity']
len(data[data['Deuda_FFPP'].isnull()])

4977

### Deuda corto plazo / FFPP

In [13]:
data['DeudaCP_FFPP'] = data['Short Term Debt'] / data['Total Equity']
len(data[data['DeudaCP_FFPP'].isnull()])

4611

## Variaciones en los ratios

A continuación vamos a incorporar como variable la variación de cada ratio con respecto al periodo anterior. Para ello primero aseguramos el orden adecuado de la tabla

In [14]:
data.sort_values(['Ticker','Fiscal Year','Fiscal Period'], ascending=True, inplace=True)

Despues incorporamos en cada fila el valor del ratio del periodo anterior, siempre que el Ticker no varíe.

In [15]:
#Definimos una función que genere subtablas por Ticker y en ellas haga un shift para incorporar en cada línea el
#valor de un ratio dado del periodo anterior.

def ratio_ant(ratio,data):
    table_1=pd.DataFrame()
    for ticker in data.Ticker.unique():
        aux_table=data[data.Ticker==ticker]
        aux_table['var_%s'%(ratio)]=(aux_table[ratio].shift(1)/aux_table[ratio])-1
        table_1=table_1.append(aux_table)
    return table_1

In [16]:
#Aplicamos la función anterior para cada ratio

for column in ['PER','PBC','ROE','ROA','Deuda_FFPP','DeudaCP_FFPP']:
    data=ratio_ant(column,data)

## Valor relativo de cada ratio

Una posible mejora del modelo consiste en considerar cada uno de los ratios no por su valor absoluto, sino como un valor relativo con respecto al resto de compañias del periodo.
Para ello vamos a obtener para cada periodo el percentil que ocupa cada ratio de cada compañía con respecto a las demás.

In [17]:
data.head(4)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Sector,Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Periodo,Price_d,Price_d_100,Publish_Date+180,Price_d+180,Price_d+180_100,Indice_d_100,Indice_d+180_100,Var_cotiz,Var_index,Etiqueta,Capit,PER,PBC,ROE,ROA,Deuda_FFPP,DeudaCP_FFPP,var_PER,var_PBC,var_ROE,var_ROA,var_Deuda_FFPP,var_DeudaCP_FFPP
4567,AAL,68568,USD,2013,Q4,2013-12-31,2014-02-28,2015-02-25,372054000.0,293814000.0,9.251000e+09,1.560000e+09,1.012000e+09,1.432300e+10,1.925900e+10,NaN,8.696000e+09,2.795500e+10,42278000000,4.975000e+09,1.446000e+09,1.380600e+10,1.535300e+10,3.120300e+10,45009000000,1.059700e+10,NaN,-1.129600e+10,-2.731000e+09,42278000000,Industrials,7.368000e+09,-5.455000e+09,1.913000e+09,-1.798000e+09,-301000000.0,NaN,-238000000.0,1.150000e+08,-221000000.0,-209000000.0,-1.060000e+08,-2.220000e+09,-2.326000e+09,324000000.0,-2.002000e+09,NaN,-2.002000e+09,-2.002000e+09,2013Q4,34.81,187.553879,2014-08-27,36.85,198.545259,119.877093,124.142159,0.058604,0.035579,Mejor,1.295120e+10,-6.469131,-4.742292,0.733065,-0.047353,-6.151227,-0.529476,NaN,NaN,NaN,NaN,NaN,NaN
4568,AAL,68568,USD,2014,Q1,2014-03-31,2014-04-24,2014-04-24,723971000.0,741335000.0,9.664000e+09,2.008000e+09,1.052000e+09,1.519300e+10,1.994400e+10,NaN,8.600000e+09,2.854400e+10,43737000000,5.001000e+09,1.441000e+09,1.507900e+10,1.524400e+10,2.756300e+10,42642000000,1.404600e+10,NaN,-1.081600e+10,1.095000e+09,43737000000,Industrials,9.995000e+09,-7.653000e+09,2.342000e+09,-1.612000e+09,-401000000.0,NaN,-307000000.0,7.300000e+08,-237000000.0,-236000000.0,4.930000e+08,NaN,4.930000e+08,-13000000.0,4.800000e+08,NaN,4.800000e+08,4.800000e+08,2014Q1,35.12,189.224138,2014-10-21,35.77,192.726293,121.211646,116.720136,0.018508,-0.037055,Mejor,2.542586e+10,52.970545,23.219965,0.438356,0.010975,15.237443,1.315982,-1.122127,-1.204233,0.672304,-5.314767,-1.403692,-1.402343
4569,AAL,68568,USD,2014,Q2,2014-06-30,2014-07-24,2014-07-24,720600000.0,734767000.0,9.459000e+09,1.981000e+09,1.093000e+09,1.496600e+10,2.123600e+10,NaN,8.609000e+09,2.984500e+10,44811000000,5.003000e+09,1.523000e+09,1.508800e+10,1.520500e+10,2.563700e+10,40725000000,1.588600e+10,-1000000.0,-9.953000e+09,4.086000e+09,44811000000,Industrials,1.135500e+10,-7.917000e+09,3.438000e+09,-2.039000e+09,-402000000.0,NaN,-319000000.0,1.399000e+09,-195000000.0,-206000000.0,1.204000e+09,NaN,1.204000e+09,-340000000.0,8.640000e+08,NaN,8.640000e+08,8.640000e+08,2014Q2,39.74,214.116379,2015-01-20,49.97,269.234914,122.811785,122.548804,0.257423,-0.002141,Mejor,2.863664e+10,33.144264,7.008479,0.211454,0.019281,4.093979,0.372736,0.598181,2.313125,1.073059,-0.430802,2.721915,2.530598
4570,AAL,68568,USD,2014,Q3,2014-09-30,2014-10-23,2014-10-23,719067000.0,735196000.0,7.899000e+09,1.961000e+09,1.182000e+09,1.345000e+10,2.216300e+10,NaN,8.560000e+09,3.072300e+10,44173000000,4.626000e+09,1.439000e+09,1.388800e+10,1.565100e+10,2.531400e+10,39202000000,1.595000e+10,NaN,-9.086000e+09,4.971000e+09,44173000000,Industrials,1.113900e+10,-7.900000e+09,3.239000e+09,-1.979000e+09,-393000000.0,NaN,-3340

In [18]:
data = data.astype({'Fiscal Year':str})
data['Periodo'] = data['Fiscal Year'] + data['Fiscal Period']

In [19]:
len(data)

12706

In [20]:
#Definimos una función que genere subtablas por periodo y cada una calcule el cuantil de cada ratio de cada
#compañia

In [21]:
from scipy import stats

def my_quantile(ratio,data):
    table_1=pd.DataFrame()
    
    for periodo in data.Periodo.unique():
        aux_table=data[data.Periodo==periodo]
        aux_table['quantile_%s'%(ratio)]=0       
        
        for row in range(0,len(aux_table)):
            aux_table['quantile_%s'%(ratio)].iloc[row]=stats.percentileofscore(aux_table[ratio],
                                                                               aux_table[ratio].iloc[row])
        
        table_1=table_1.append(aux_table)
            
    return table_1

In [22]:
#Aplicamos la función anterior para cada ratio

for column in ['PER','PBC','ROE','ROA','Deuda_FFPP','DeudaCP_FFPP']:
    data=my_quantile(column,data)

## Variaciones en los cuantiles

A continuación vamos a incorporar como variable la variación de cada percentil con respecto al periodo anterior. Para ello primero aseguramos el orden adecuado de la tabla

In [23]:
data.sort_values(['Ticker','Fiscal Year','Fiscal Period'], ascending=True, inplace=True)

Despues incorporamos en cada fila el valor del ratio del periodo anterior, siempre que el Ticker no varíe.

In [24]:
#Definimos una función que genere subtablas por Ticker y en ellas haga un shift para incorporar en cada línea el
#valor de un cuantil dado del periodo anterior.

def ratio_ant(ratio,data):
    table_1=pd.DataFrame()
    for ticker in data.Ticker.unique():
        aux_table=data[data.Ticker==ticker]
        aux_table['var_%s'%(ratio)]=(aux_table[ratio]-aux_table[ratio].shift(1))
        table_1=table_1.append(aux_table)
    return table_1

In [25]:
#Aplicamos la función anterior para cada ratio

for column in ['quantile_PER','quantile_PBC','quantile_ROE','quantile_ROA','quantile_DeudaCP_FFPP']:
    data=ratio_ant(column,data)

In [26]:
data[data.Ticker=='MRO'].head(10)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Sector,Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Periodo,Price_d,Price_d_100,Publish_Date+180,Price_d+180,Price_d+180_100,Indice_d_100,Indice_d+180_100,Var_cotiz,Var_index,Etiqueta,Capit,PER,PBC,ROE,ROA,Deuda_FFPP,DeudaCP_FFPP,var_PER,var_PBC,var_ROE,var_ROA,var_Deuda_FFPP,var_DeudaCP_FFPP,quantile_PER,quantile_PBC,quantile_ROE,quantile_ROA,quantile_Deuda_FFPP,quantile_DeudaCP_FFPP,var_quantile_PER,var_quantile_PBC,var_quantile_ROE,var_quantile_ROA,var_quantile_DeudaCP_FFPP
805,MRO,378185,USD,2013,Q4,2013-12-31,2014-02-28,2015-03-02,695000000.0,699000000.0,2.640000e+08,2.134000e+09,364000000.0,2.975000e+09,2.814500e+10,1.201000e+09,3.299000e+09,3.264500e+10,35620000000,3.891000e+09,2.030000e+08,4.333000e+09,6.394000e+09,1.194300e+10,16276000000,7.362000e+09,-2.903000e+09,1.513500e+10,1.934400e+10,35620000000,Energy,1.646000e+09,-500000000.0,1.146000e+09,-1.564000e+09,-675000000.0,-220000000.0,-497000000.0,-418000000.0,-68000000.0,-68000000.0,-486000000.0,-47000000.0,-5.330000e+08,447000000.0,-86000000.0,4.610000e+08,3.750000e+08,3.750000e+08,2013Q4,30.05,96.654873,2014-08-27,37.08,119.266645,109.637285,124.523691,0.233943,0.135779,Mejor,2.088475e+10,55.692667,1.079650,0.019386,0.010528,0.341036,0.010494,NaN,NaN,NaN,NaN,NaN,NaN,45.383104,10.216110,42.043222,44.597250,23.575639,20.039293,NaN,NaN,NaN,NaN,NaN
806,MRO,378185,USD,2014,Q1,2014-03-31,2014-05-07,2014-05-07,693000000.0,696000000.0,1.964000e+09,2.222000e+09,405000000.0,4.787000e+09,2.842600e+10,1.223000e+09,1.715000e+09,3.136400e+10,36151000000,4.038000e+09,6.800000e+07,4.314000e+09,6.392000e+09,1.203200e+10,16346000000,7.369000e+09,-3.445000e+09,1.615100e+10,1.980500e+10,36151000000,Energy,2.849000e+09,-542000000.0,2.307000e+09,-1.643000e+09,-729000000.0,-73000000.0,-643000000.0,664000000.0,-49000000.0,-49000000.0,615000000.0,-17000000.0,5.980000e+08,-200000000.0,398000000.0,7.510000e+08,1.149000e+09,1.149000e+09,2014Q1,31.87,102.508845,2014-11-03,31.35,100.836282,119.151929,104.228790,-0.016316,-0.125245,Mejor,2.208591e+10,19.221854,1.115168,0.058016,0.031783,0.326180,0.003433,1.897362,-0.031850,-0.665851,-0.668764,0.045545,2.056439,32.593857,13.481229,88.225256,88.907850,23.549488,11.262799,-12.789247,3.265119,46.182034,44.310600,-8.776494
807,MRO,378185,USD,2014,Q2,2014-06-30,2014-08-07,2014-08-07,676000000.0,679000000.0,1.169000e+09,2.042000e+09,404000000.0,4.218000e+09,2.782400e+10,1.184000e+09,2.709000e+09,3.171700e+10,35935000000,2.753000e+09,6.800000e+07,3.974000e+09,6.362000e+09,1.209300e+10,16067000000,7.300000e+09,-3.718000e+09,1.656400e+10,1.986800e+10,35935000000,Energy,2.941000e+09,-562000000.0,2.379000e+09,-1.788000e+09,-753000000.0,-145000000.0,-680000000.0,591000000.0,-76000000.0,-76000000.0,515000000.0,-4000000.0,5.110000e+08,-151000000.0,360000000.0,1.800000e+08,5.400000e+08,5.400000e+08,2014Q2,34.35,110.485687,2015-02-03,26.28,84.528787,119.150581,95.930014,-0.234934,-0.194884,Peor,2.322060e+10,43.001111,1.168744,0.027179,0.015027,0.323636,0.003423,-0.552992,-0.045840,1.13

In [27]:
data.to_csv('../tablas/dataformodel.csv', index=False)